In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [2]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15, poll_frequency=1)
base_url = 'https://www.airbnb.co.in/s/New-Delhi--India/homes?place_id=ChIJLbZ-NFv9DDkRzk0gTkm3wlI&refinement_paths%5B%5D=%2Fhomes'
driver.get(base_url)
driver.maximize_window()

In [3]:
num_pages = 15
all_data = []

In [4]:
for _ in range(num_pages):
    try:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.lxq01kf")))

        all_items = driver.find_elements(By.CSS_SELECTOR, "div.lxq01kf")
        if not all_items:
            break
        
        for item in all_items:
            name = item.find_element(By.CSS_SELECTOR, "div.t1jojoys").text.strip() if item.find_elements(By.CSS_SELECTOR, "div.t1jojoys") else ''
            description = item.find_element(By.CSS_SELECTOR, "span.t6mzqp7").text.strip() if item.find_elements(By.CSS_SELECTOR, "span.t6mzqp7") else ''
            rating_reviews = item.find_element(By.CSS_SELECTOR, "span.r4a59j5").text.strip() if item.find_elements(By.CSS_SELECTOR, "span.r4a59j5") else '' 

            if "out" in rating_reviews:  
                rating = rating_reviews.split(" out")[0]
                reviews = rating_reviews.split(",")[-1].strip().split(" ")[0] 
            else: 
                rating = '0'
                reviews = '0'  
            price = item.find_element(By.CSS_SELECTOR, "span._11jcbg2").text.strip().replace("₹", "").strip() if item.find_elements(By.CSS_SELECTOR, "span._11jcbg2") else '' 

            # Extract place name from the name (assumes last word in name is the place)
            place = name.split()[-1] if name else ''

            all_data.append({
                'Name': name,
                'Description': description,
                'Price/Night': price,
                'Rating': rating,
                'Reviews': reviews,
                'Place': place  
            })

        # Handling pagination by clicking the "Next" button
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[aria-label='Next']")))
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            
            if 'disabled' not in next_button.get_attribute('class'):
                next_button.click()
                time.sleep(7) 
            else:
                break 

        except Exception:
            break 

    except Exception:
        break

driver.quit()


In [5]:
df = pd.DataFrame(all_data)
df

,Name,Description,Price/Night,Rating,Reviews,Place
0,Guest suite in New Delhi,room with private entrance in New Delhi GK1 +GYM,"2,066",4.92,131,Delhi
1,Flat in Gurugram,Stylish 1 BHK Serviced Apartment Near Cyber Hub,"2,640",4.93,731,Gurugram
2,Place to stay in New Delhi,DD homestay 3 (international) near all monuments,"2,123",5.0,38,Delhi
3,Apartment in Delhi,Shubhvir Paradise | Studio Apartment in West D...,"2,626",4.88,132,Delhi
4,Room in Delhi,Spacious room with private balcony and pantry,"1,500",4.97,36,Delhi
...,...,...,...,...,...,...
265,Flat in Noida,5 Star Apartment at Sky Heights - Love Lounge,"3,332",0,0,Noida
266,Place to stay in New Delhi,UMR Private Room @ Hauz Khas Village,"2,069",4.77,61,Delhi
267,Flat in New Delhi,Peaceful Park View Apartment near Delhi Airport,"5,706",4.92,232,Delhi
268,Place to stay in New Delhi,Zen Retreat Pvt BR w/KingBed/Living/Patio+Swing,"2,431",4.83,6,Delhi


In [7]:
df.to_csv('airbnb_Delhi_listings.csv', index=False, encoding='utf-8')